# Vibration Analysis
* Author: Steve Hanly, Endaq [blog.endaq.com/...](https://blog.endaq.com/top-vibration-metrics-to-monitor-how-to-calculate-them?)
* Data: "Bearing Vibration Data" by Case Western Reserve Bearing Data Center [info.endaq.com/...](https://info.endaq.com/hubfs/Plots/bearing_data.csv)
* Adapted by: Juan Diego Cardenas Cartagena

## Bearing Vibration Data


In this tutorial, we will discuss various techniques for analyzing the vibrations of a bearing in the testing system, as shown below, with a particular focus on spectrum-domain techniques. The dataset describes the acceleration in g units ($1g  \approx 9.81 m/s^2$) of bearings under different conditions. For each dataset, the naming convention indicates the fault type. For example, **Fault_0014** represents a fan-end bearing fault in the inner race with a defect measuring 0.014 inches in diameter. Figure 1 shows the components of a bearing.

<figure>
<center>
<img src="img/bearing.jpg" />
<figcaption>Figure 1: Bearing components - Faults in the dataset are in the inner race. Image by G. Fonts</figcaption>
</center>
</figure>

The test stand, as shown in Figure 2, includes a 2 hp motor (left), a torque transducer/encoder (center), a dynamometer (right), and control electronics. Test bearings supporting the motor shaft were intentionally damaged using electro-discharge machining to introduce faults with diameters of 0.007", 0.014", 0.021", and others. SKF bearings were used for those small faults. The tests were conducted with the motor operating at 1772 RPM (29.5 Hz).

<figure>
<center>
<img src="img/testing-system.png" />
<figcaption>Figure 2: Testing system. Image taken from Apparatus & Procedures - Bearing Data Center</figcaption>
</center>
</figure>

Vibration data were collected using accelerometers mounted on the motor housing and sometimes the base plate. Signals were recorded by a 16-channel Digital Audio Tape (DAT) recorder and processed in Matlab. Sampling rates were 12,000 Hz and, for some tests, 48,000 Hz. Speed and horsepower data were recorded via the torque transducer.

For further details about the dataset and experiment, please go to [Case Western Reserve University Bearing Data Center](https://engineering.case.edu/bearingdatacenter/welcome)

## Import Dependencies and Upload Data

In [38]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.fft as fft
import scipy.signal as signal

# Load the bearing vibration data
data_path = 'data/bearing_data.csv'
bearing_data = pd.read_csv(data_path)

# Show the first few rows of the data
print(bearing_data.head())

# Extract the data as a numpy array
time = bearing_data['Time'].values
normal_vibration = bearing_data['Normal'].values
fault_007_vibration = bearing_data['Fault_007'].values
fault_014_vibration = bearing_data['Fault_014'].values
fault_021_vibration = bearing_data['Fault_021'].values

       Time  Fault_021  Fault_014  Fault_007    Normal
0  0.000000  -0.105351  -0.074395   0.053116  0.046104
1  0.000083   0.132888   0.056365   0.116628 -0.037134
2  0.000167  -0.056535   0.201257   0.083654 -0.089496
3  0.000250  -0.193178  -0.024528  -0.026477 -0.084906
4  0.000333   0.064879  -0.072284   0.045319 -0.038594


## Time-domain Analysis

### Visual Inspection and Statistical Metrics

In [39]:
# Plot 0.3 second of vibration data
mask = (time >= 3.7) & (time <= 4.0) # 0.3 seconds of data

fig = px.line(x=time[mask], y=[fault_021_vibration[mask], fault_014_vibration[mask], fault_007_vibration[mask], normal_vibration[mask]], labels={'x': 'Time (s)', 'value': 'Acceleration (g)'}, title='Normal and Fault Vibration Data - Time Domain')
fig.update_layout(legend_title_text='Vibration Type')
fig.update_traces(mode='lines')
fig.data[0].name = 'Fault_021'
fig.data[1].name = 'Fault_014'
fig.data[2].name = 'Fault_007'
fig.data[3].name = 'Normal'
fig.show()


Let us begin by analyzing the **peak** for each case which gives us a notion of the signal's maximum amplitude. The peak is defined as
$$
\text{peak} = \max(|x(t)|),
$$
where $x(t)$ is the vibration signal and $t>0$.

In [40]:
# Compute maximum and minimum acceleration for each type of vibration
normal_peak = np.max(np.abs(normal_vibration))
fault_007_peak = np.max(np.abs(fault_007_vibration))
fault_014_peak = np.max(np.abs(fault_014_vibration))
fault_021_peak = np.max(np.abs(fault_021_vibration))

# Print the results
print('Normal Peak Acceleration: {:.3f} g'.format(normal_peak))
print('Fault 007 Peak Acceleration: {:.3f} g'.format(fault_007_peak))
print('Fault 014 Peak Acceleration: {:.3f} g'.format(fault_014_peak))
print('Fault 021 Peak Acceleration: {:.3f} g'.format(fault_021_peak))

Normal Peak Acceleration: 0.269 g
Fault 007 Peak Acceleration: 0.650 g
Fault 014 Peak Acceleration: 1.339 g
Fault 021 Peak Acceleration: 1.038 g


Now, let us compute the **Root Mean Square (RMS)** for each case. The RMS provides a measure of the vibration magnitude and offers insight into the energy content of the signal. The RMS in finite-length signal $x[n] = \{x[0], x[1], x[2], ..., x[N-1]\}$ is given by
$$ x_{\text{RMS}} = \sqrt{\frac{1}{n} \sum_{i = 0}^{N-1} x[i]^{2}}. $$


In [41]:
# Compute the RMS acceleration for each type of vibration
normal_rms = np.sqrt(np.mean(normal_vibration**2))
fault_007_rms = np.sqrt(np.mean(fault_007_vibration**2))
fault_014_rms = np.sqrt(np.mean(fault_014_vibration**2))
fault_021_rms = np.sqrt(np.mean(fault_021_vibration**2))

# Print the results
print('Normal Vibration: RMS = {:.3f} g'.format(normal_rms))
print('Fault_007 Vibration: RMS = {:.3f} g'.format(fault_007_rms))
print('Fault_014 Vibration: RMS = {:.3f} g'.format(fault_014_rms))
print('Fault_021 Vibration: RMS = {:.3f} g'.format(fault_021_rms))

Normal Vibration: RMS = 0.066 g
Fault_007 Vibration: RMS = 0.121 g
Fault_014 Vibration: RMS = 0.158 g
Fault_021 Vibration: RMS = 0.199 g


With the RMS and peak values for each signal, let us compute the **Crest Factor**, which quantifies the sharpness of peaks or how "peaky" a signal is. As a reference, impulse signals typically exhibit a higher crest factor compared to smoother signals. The Crest Factor is defined as:
$$
\text{Crest Factor} = \frac{\text{Peak}}{\text{RMS}}.
$$

In [42]:
# Compute the crest factor for each type of vibration
normal_crest = normal_peak / normal_rms
fault_007_crest = fault_007_peak / fault_007_rms
fault_014_crest = fault_014_peak / fault_014_rms
fault_021_crest = fault_021_peak / fault_021_rms

# Print the results
print('Normal Vibration: Crest Factor = {:.3f}'.format(normal_crest))
print('Fault_007 Vibration: Crest Factor = {:.3f}'.format(fault_007_crest))
print('Fault_014 Vibration: Crest Factor = {:.3f}'.format(fault_014_crest))
print('Fault_021 Vibration: Crest Factor = {:.3f}'.format(fault_021_crest))

Normal Vibration: Crest Factor = 4.081
Fault_007 Vibration: Crest Factor = 5.361
Fault_014 Vibration: Crest Factor = 8.484
Fault_021 Vibration: Crest Factor = 5.222


And finally, the **Standard Deviation** gives us a notion of variation in the signal. The Standard Deviation in a finite-length signal $x[n] = \{x[0], x[1], x[2], ..., x[N-1]\}$ is defined as:
$$
\sigma = \sqrt{\frac{1}{N-1} \sum_{i = 0}^{N-1} (x[i] - \mu)^{2}},
$$
where the mean $\mu$ is given by
$$
\mu = \frac{1}{N} \sum_{i = 0}^{N-1} x[i].
$$

In [43]:
# Compute the standard deviation of the acceleration for each type of vibration
normal_std = np.std(normal_vibration)
fault_007_std = np.std(fault_007_vibration)
fault_014_std = np.std(fault_014_vibration)
fault_021_std = np.std(fault_021_vibration)

# Print the results
print('Normal Vibration: Standard Deviation = {:.3f} g'.format(normal_std))
print('Fault_007 Vibration: Standard Deviation = {:.3f} g'.format(fault_007_std))
print('Fault_014 Vibration: Standard Deviation = {:.3f} g'.format(fault_014_std))
print('Fault_021 Vibration: Standard Deviation = {:.3f} g'.format(fault_021_std))

Normal Vibration: Standard Deviation = 0.065 g
Fault_007 Vibration: Standard Deviation = 0.121 g
Fault_014 Vibration: Standard Deviation = 0.158 g
Fault_021 Vibration: Standard Deviation = 0.198 g


### Question for you: 
1. What can you conclude from this information from the time-domain analysis? 
2. Do you notice any relevant feature or threshold to evaluate the conditions of a bearing?

## Power Spectral Density (PSD) Analysis

The Power Spectral Density (PSD) of a signal quantifies how the signal's power is distributed across various frequencies. In this tutorial, we will estimate the PSD of the vibration data using Welch's method. For further information about this method, visit Smith, J.O. "Welch's Method", in Spectral Audio Signal Processing, [ccrma.stanford.edu/~jos/sasp/Welch_s_Method.html](https://ccrma.stanford.edu/~jos/sasp/Welch_s_Method.html), 2011 edition.

For reference, the average power of a discrete signal $ x[n] = \{x[0], x[1], x[2], \dots, x[N-1]\} $ in the time domain is given by:  
$$
P_{\text{avg}} = \frac{1}{N} \sum_{i = 0}^{N-1} \lvert x[n] \rvert^2.
$$
This value represents the average energy per sample over the signal's duration.  

Another relevant quantity for this discussion is the instantaneous power, defined as $ P[n] = \lvert x[n] \rvert^2, \, n = 0, 1, \dots, N-1 $, which indicates the signal's power at a specific point in time. 

**Note:** In this tutorial, we will not explore how to transform these power-related quantities from the time domain to the frequency domain. For those curious, this process involves using the Fast Fourier Transform (FFT), a topic we will cover in the coming weeks.

In [44]:
# Compute the PSD of the vibration data via Welch's method
fs =len(time) / (time[-1] - time[0])
bin_width = 1
f, Pxx_normal = signal.welch(normal_vibration, fs, nperseg=fs/bin_width, window='hann')
f, Pxx_fault_007 = signal.welch(fault_007_vibration, fs, nperseg=fs/bin_width, window='hann')
f, Pxx_fault_014 = signal.welch(fault_014_vibration, fs, nperseg=fs/bin_width, window='hann')
f, Pxx_fault_021 = signal.welch(fault_021_vibration, fs, nperseg=fs/bin_width, window='hann')

# Plot the PSD of the vibration data in log-log scale
fig = px.line(x=f, y=[Pxx_normal, Pxx_fault_007, Pxx_fault_014, Pxx_fault_021], labels={'x': 'Frequency (Hz)', 'value': 'PSD (g^2/Hz)'}, title='Normal and Fault Vibration Data - Frequency Domain')
fig.update_layout(legend_title_text='Vibration Type', xaxis_type="log", yaxis_type="log")
fig.update_traces(mode='lines')
fig.data[0].name = 'Normal'
fig.data[1].name = 'Fault_007'
fig.data[2].name = 'Fault_014'
fig.data[3].name = 'Fault_021'
fig.show()

Let us now analyze key information from the signal in the frequency domain, including the Peak and RMS values.

In [45]:
# Compute the Peak Frequency of the PSD for each type of vibration
normal_peak_freq = f[np.argmax(Pxx_normal)]
fault_007_peak_freq = f[np.argmax(Pxx_fault_007)]
fault_014_peak_freq = f[np.argmax(Pxx_fault_014)]
fault_021_peak_freq = f[np.argmax(Pxx_fault_021)]

# Compute the Peak amplitude of the PSD for each type of vibration
normal_peak_mag = np.max(Pxx_normal)
fault_007_peak_mag = np.max(Pxx_fault_007)
fault_014_peak_mag = np.max(Pxx_fault_014)
fault_021_peak_mag = np.max(Pxx_fault_021)

# Print the results
print('Normal Vibration: Peak Frequency = {:.3f} Hz, Peak Magnitude = {:.6f} g^2/Hz'.format(normal_peak_freq, normal_peak_mag))
print('Fault_007 Vibration: Peak Frequency = {:.3f} Hz, Peak Magnitude = {:.6f} g^2/Hz'.format(fault_007_peak_freq, fault_007_peak_mag))
print('Fault_014 Vibration: Peak Frequency = {:.3f} Hz, Peak Magnitude = {:.6f} g^2/Hz'.format(fault_014_peak_freq, fault_014_peak_mag))
print('Fault_021 Vibration: Peak Frequency = {:.3f} Hz, Peak Magnitude = {:.6f} g^2/Hz'.format(fault_021_peak_freq, fault_021_peak_mag))

Normal Vibration: Peak Frequency = 90.000 Hz, Peak Magnitude = 0.000473 g^2/Hz
Fault_007 Vibration: Peak Frequency = 1051.000 Hz, Peak Magnitude = 0.000600 g^2/Hz
Fault_014 Vibration: Peak Frequency = 3237.000 Hz, Peak Magnitude = 0.001713 g^2/Hz
Fault_021 Vibration: Peak Frequency = 3316.000 Hz, Peak Magnitude = 0.001796 g^2/Hz


The RMS in the PSD is given by
$$
g_{\text{RMS}} = \frac{1}{N} \sum_{k = f_1}^{f_2} \lvert X[k] \rvert^2,
$$
where $f_1$ and $f_2$ are the lower and upper bound frequencies in the signal, respectively, and $X[f]$ is the frequency-domain representation of the signal $x[n]$.

In [46]:
# Compute the RMS of the PSD for each type of vibration
normal_rms_psd = np.sqrt(np.trapz(Pxx_normal, f))
fault_007_rms_psd = np.sqrt(np.trapz(Pxx_fault_007, f))
fault_014_rms_psd = np.sqrt(np.trapz(Pxx_fault_014, f))
fault_021_rms_psd = np.sqrt(np.trapz(Pxx_fault_021, f))

# Print the results
print('Normal Vibration: RMS PSD = {:.3f} g'.format(normal_rms_psd))
print('Fault_007 Vibration: RMS PSD = {:.3f} g'.format(fault_007_rms_psd))
print('Fault_014 Vibration: RMS PSD = {:.3f} g'.format(fault_014_rms_psd))
print('Fault_021 Vibration: RMS PSD = {:.3f} g'.format(fault_021_rms_psd))

Normal Vibration: RMS PSD = 0.065 g
Fault_007 Vibration: RMS PSD = 0.122 g
Fault_014 Vibration: RMS PSD = 0.157 g
Fault_021 Vibration: RMS PSD = 0.199 g


In [47]:
# Compute how the RMS of the PSD builds up over the frequency range
normal_rms_psd_cumulative = np.sqrt(np.cumsum(Pxx_normal))
fault_007_rms_psd_cumulative = np.sqrt(np.cumsum(Pxx_fault_007))
fault_014_rms_psd_cumulative = np.sqrt(np.cumsum(Pxx_fault_014))
fault_021_rms_psd_cumulative = np.sqrt(np.cumsum(Pxx_fault_021))

# Plot the cumulative RMS of the PSD with log scale in x-axis
fig = px.line(x=f, y=[normal_rms_psd_cumulative, fault_007_rms_psd_cumulative, fault_014_rms_psd_cumulative, fault_021_rms_psd_cumulative], labels={'x': 'Frequency (Hz)', 'value': 'Cumulative RMS PSD (g)'}, title='Normal and Fault Vibration Data - Cumulative RMS PSD')
fig.update_layout(legend_title_text='Vibration Type', xaxis_type="log")
fig.update_traces(mode='lines')
fig.data[0].name = 'Normal'
fig.data[1].name = 'Fault_007'
fig.data[2].name = 'Fault_014'
fig.data[3].name = 'Fault_021'
fig.show()

## Summary

In [ ]:
# Create a dictionary with the time-domain results
time_domain_results = {
    'Normal': [normal_peak, normal_rms, normal_crest, normal_std],
    'Fault_007': [fault_007_peak, fault_007_rms, fault_007_crest, fault_007_std],
    'Fault_014': [fault_014_peak, fault_014_rms, fault_014_crest, fault_014_std],
    'Fault_021': [fault_021_peak, fault_021_rms, fault_021_crest, fault_021_std],  
}

# Create a DataFrame from the time-domain dictionary
time_domain_df = pd.DataFrame(time_domain_results, index=['Peak Acceleration (g)', 'RMS Acceleration (g)', 'Crest Factor', 'Standard Deviation (g)'])

# Print the time-domain results
print("Time-Domain Results:")
print(time_domain_df)

# Create a dictionary with the frequency-domain results
frequency_domain_results = {
    'Normal': [normal_peak_freq, normal_peak_mag, normal_rms_psd],
    'Fault_007': [fault_007_peak_freq, fault_007_peak_mag, fault_007_rms_psd],
    'Fault_014': [fault_014_peak_freq, fault_014_peak_mag, fault_014_rms_psd],
    'Fault_021': [fault_021_peak_freq, fault_021_peak_mag, fault_021_rms_psd],  
}

# Create a DataFrame from the frequency-domain dictionary
frequency_domain_df = pd.DataFrame(frequency_domain_results, index=['Peak Frequency (Hz)', 'Peak Magnitude (g^2/Hz)', 'RMS PSD (g)'])

# Print the frequency-domain results
print("\nFrequency-Domain Results:")
print(frequency_domain_df)

Time-Domain Results:
                          Normal  Fault_007  Fault_014  Fault_021
Peak Acceleration (g)   0.269114   0.650390   1.338628   1.037862
RMS Acceleration (g)    0.065943   0.121308   0.157784   0.198760
Crest Factor            4.081014   5.361487   8.483951   5.221685
Standard Deviation (g)  0.065060   0.121272   0.157760   0.198382

Frequency-Domain Results:
                            Normal    Fault_007    Fault_014    Fault_021
Peak Frequency (Hz)      90.000000  1051.000000  3237.000000  3316.000000
Peak Magnitude (g^2/Hz)   0.000473     0.000600     0.001713     0.001796
RMS PSD (g)               0.064886     0.121984     0.157235     0.198519


### Questions for you: 
1. Do you notice any relevant pattern between normal and faulty bearings? 
2. Is there any relation between the time-domain and frequency-domain values?
3. How would you design a data-driven decision support system for predictive maintenance? Are there additional features you would consider including?
4. Can you think of another application of this vibration analysis?
5. Why is the RMS different in the time and frequency domains? Shouldn’t they be equal?